Before proceeding, please ensure that your notebook is connected to a GPU runtime.

In [ ]:
%cd /content
!git clone https://github.com/atakan-topaloglu/2d-gaussian-splatting --recursive
!pip install -q plyfile

In [ ]:
# --- 终极环境设置脚本 ---

# 步骤 1: 安装完整的 CUDA 11.8 工具包
print("--- 步骤 1: 安装 CUDA 11.8 Toolkit (这可能需要几分钟) ---")
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-keyring_1.1-1_all.deb
!dpkg -i cuda-keyring_1.1-1_all.deb
!apt-get update
!apt-get -y install cuda-toolkit-11-8

# 步骤 2: 设置环境变量，强制使用 CUDA 11.8
print("\n--- 步骤 2: 设置环境变量指向 CUDA 11.8 ---")
import os
os.environ['CUDA_HOME'] = '/usr/local/cuda-11.8'
os.environ['PATH'] = '/usr/local/cuda-11.8/bin:' + os.environ['PATH']
# 验证 nvcc 版本
!nvcc --version

# 步骤 3: 安装并设置兼容的 C++ 编译器 (g++-10)
print("\n--- 步骤 3: 安装并设置 g++-10 ---")
!apt-get -y install g++-10
!update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-10 10
!update-alternatives --set g++ /usr/bin/g++-10
# 验证 g++ 版本
!g++ --version

# 步骤 4: 安装与新环境完全匹配的 PyTorch
print("\n--- 步骤 4: 安装 PyTorch for CUDA 11.8 ---")
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118

print("\n\n--- ✅ 环境设置完成！ ---")

In [ ]:
%cd /content/2d-gaussian-splatting/

In [ ]:
import os
os.environ['CUDA_HOME'] = '/usr/local/cuda' #ensure that the CUDA path is correct
!pip install ninja
!pip install submodules/simple-knn
!pip install submodules/diff-surfel-rasterization

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/2d-gaussian-splatting/

For Colab to access your COLMAP processed data, please copy the COLMAP output to your Google Drive and enter the full COLMAP source path (e.g. /content/drive/MyDrive/COLMAP_out)

In [ ]:
COLMAP_PATH = str(input('Please enter COLMAP output path:'))

In [ ]:
!python train.py -s $COLMAP_PATH

Training is complete, to use render.py, please enter the desired model_path located in your Drive: /content/drive/MyDrive/2d_gaussian_splatting/output/{10 digit UUID}: (e.g: /content/drive/MyDrive/2d_gaussian_splatting/output/8d812792-1)

In [ ]:
MODEL_PATH = str(input('Please enter the model path:'))

In [ ]:
!pip3 install open3d==0.18.0 mediapy==1.1.2 lpips==0.1.4 scikit-image==0.21.0 tqdm==4.66.2 trimesh==4.3.2 opencv-python

In [ ]:
!python render.py -m $MODEL_PATH -s $COLMAP_PATH

Training and rendering is complete, please navigate to your drive and see the model outputs under the directory "2d_gaussian_splatting/".